In [51]:
import pandas as pd 
import glob 
import re
import numpy as np
import ast

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
df=pd.read_csv("dfscopus-1.csv")
df.columns

Index(['authors', 'author(s) id', 'title', 'year', 'source title', 'volume',
       'issue', 'art. no.', 'page start', 'page end', 'page count', 'cited by',
       'doi', 'link', 'affiliations', 'authors with affiliations', 'abstract',
       'author keywords', 'index keywords', 'funding details',
       'correspondence address', 'publisher', 'pubmed id', 'access type'],
      dtype='object')

In [18]:
#delete some columns we won't need 
del df["volume"]
del df["issue"]
del df["art. no."]
del df["page count"]
del df["doi"]
del df["link"]
del df["authors with affiliations"]
del df["publisher"]
del df["pubmed id"]

In [19]:
df.head(20)

,authors,author(s) id,title,year,source title,page start,page end,cited by,affiliations,abstract,author keywords,index keywords,funding details,correspondence address,access type
0,"Bonomi L., Marchini G., Marraffa M., Bernardi ...",16438564300;7005720033;6701396773;57192481974;...,Vascular risk factors for primary open angle g...,2000,Ophthalmology,1287,1293,457.0,"Clinica Oculistica, Università degli Studi di ...",Objective: To assess the impact of vascular ri...,NaN,adult; age distribution; aged; article; correl...,NaN,"Bonomi, L.; Clin. Oculistica dell'Univ. Verona...",NaN
1,"Tanure M.A.G., Cohen E.J., Sudesh S., Rapuano ...",6506659416;7403588976;6603349480;7005117622;24...,Spectrum of fungal keratitis at Wills Eye Hosp...,2000,Cornea,307,312,257.0,"Cornea Service, Wills Eye Hospital, Jefferson ...",Purpose. To report the spectrum of fungal kera...,Candida albicans; Contact lens; Corneal ulcer;...,amphotericin B; fluconazole; itraconazole; ket...,NaN,"Cohen, E.J.; Cornea Service, Wills Eye Hospita...",NaN
2,"Levy R.A., Vilela V.S., Cataldo M.J., Ramos R....",35398389300;6506151613;57196699440;55437940200...,Hydroxychloroquine (HCQ) in lupus pregnancy: D...,2001,Lupus,401,404,238.0,"Hospital Universitario Pedro Ernesto, UERJ, Di...","We conducted a randomized, controlled study to...",Antimalarials; Hydroxychloroquine; Lupus pregn...,antimalarial agent; DNA antibody; double stran...,NaN,"Levy, R.A.; Hospital Universitario Pedro Ernes...",NaN
3,"Bursell S.-E., Cavallerano J.D., Cavallerano A...",26326720800;6701580100;8906005300;7004750042;6...,Stereo nonmydriatic digital-video color retina...,2001,Ophthalmology,572,585,214.0,"Beetham Eye Institute, Joslin Diabetes Center,...",Objective: To evaluate the ability to determin...,NaN,adult; aged; article; diabetic retinopathy; di...,NaN,"Aiello, L.M.; Beetham Eye Institute, Joslin Di...",NaN
4,"Nichols J.J., Marsich M.M., Nguyen M., Barr J....",56704320200;6506548363;57212857874;7202043696;...,Overnight orthokeratology,2000,Optometry and Vision Science,252,259,213.0,"Ohio State University, College of Optometry, C...",Purpose: Orthokeratology is defined as the tem...,Corneal topography; Myopia; Orthokeratology; P...,Contact lenses; Diseases; Error correction; Li...,NaN,"Nichols, J.J.; College of Optometry, Ohio Stat...",NaN
5,"Kivlin J.D., Simons K.B., Lazoritz S., Ruttum ...",24432065800;7202131239;6603798821;7003470453;,Shaken baby syndrome,2000,Ophthalmology,1246,1254,163.0,"Department of Ophthalmology, Medical College o...",Purpose: To examine the comprehensive ophthalm...,NaN,article; battered child syndrome; brain injury...,NaN,"Kivlin, J.D.; Eye Institute, 925 North 87th St...",NaN
6,"Begley C.G., Caffery B., Nichols K.K., Chalmer...",36004630000;57193083807;7103268090;7103412546;,Responses of contact lens wearers to a dry eye...,2000,Optometry and Vision Science,40,46,163.0,"Indiana Univ. School of Optometry, Bloomington...",The purpose of this study was to examine ocula...,Contact lens; Dry eye; Questionnaires; Symptom...,Contact lenses; Ophthalmology; Dry eyes; Visio...,NaN,"Begley, C.G.; Indiana University, School of Op...",NaN
7,"Kvarnström G., Jakobsson P., Lennerstrand G.",7801431617;7005912719;7006394966;,Visual screening of Swedish children: An ophth...,2001,Acta Ophthalmologica Scandinavica,240,244,162.0,"Department of Ophthalmology, Linköping Univers...",Purpose: This study describes the various opth...,Amblyopia; Children; Refractive error; Screeni...,amblyopia; ametropia; article; birth; child; c...,NaN,"Kvarnström, G.; Department of Ophthalmology, U...",NaN
8,"Pearson A.R., Soneji B., Sarvananthan N., Sanf...",57213257698;6507890776;14011793500;7801488929;,Does ethnic origin influence the incidence or ...,2000,Eye,625,628,162.0,"Eye Administration Unit, Leicester Royal Infir...","Purpose. Keratoconus affects all races, yet ve...",Ethnic origin; Incidence; Keratoconus,adolescent; adult; age distribution; article; ...,NaN,"Pearson, A.R.; Eye Administration Unit

In [20]:
df["author keywords"]=df["author keywords"].str.lower()

In [21]:
df["index keywords"]=df["index keywords"].str.lower()

In [23]:
df["source title"]=df["source title"].str.lower()

In [24]:
df.reset_index(inplace=True)

# Create easy new variables 

In [40]:
#additional non-keyword features 

#funding boolean 
df["fundingB"] = np.where(df['funding details'].notnull(), 1, 0)
df["fundingB"].sum()

1164

In [31]:
df['page start'] = df['page start'].str.replace(r'[^0-9]+', '')
df['page end'] = df['page end'].str.replace(r'[^0-9]+', '')

df["pagecount"]=pd.to_numeric(df["page end"])-pd.to_numeric(df['page start'])

In [ ]:
df["pagecount"].hist()
#must clip outliers 

In [193]:
#one hot journal title 
titlesdummies=pd.get_dummies(df["source title"]) 

In [198]:
#filtering out near zero variance features from data frame while preserving names
from sklearn.feature_selection import VarianceThreshold
selector=VarianceThreshold(.995 * (1 - .995))
selector.fit_transform(np.array(titlesdummies)).shape

def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

titlesdummiesfiltered=variance_threshold_selector(titlesdummies, .995 * (1 - .995))

df=pd.concat([df, titlesdummiesfiltered], axis=1)

(9392, 34)

In [45]:
df["open access"]=pd.get_dummies(df["access type"].str.lower()) 
del df["access type"]

# Authors

In [48]:
def getcomponentwords(colindex, splitchar='; '): 
    indexkeywords=set()
    for i in range(len(df)): 
        newset=set(str(df.iloc[i, colindex]).lower().split(splitchar))
        indexkeywords=indexkeywords.union(newset)
    return list(indexkeywords)

In [ ]:
#process authors 
#first can we create a dictionary of the authors to the author id 
df[["authors", "author(s) id"]]

In [49]:
df["authorlist"]=df["authors"].str.split(',')
df["authoridlist"]=df["author(s) id"].str.split(';')

In [69]:
#number of authors
df["authortotal"]=df["authorlist"].apply(len)

In [95]:
def removeempty(test_list): 
    try: 
        newlist=[i for i in test_list if i] 
        return newlist
    except: 
        return None

In [97]:
df["authoridlist"]=df["authoridlist"].apply(removeempty)

In [109]:
#build author dictionary 
def mergelists(list1, list2): 
    
    merged_list = [] 
    if list2 and list1: 
        for i in range(max((len(list1), len(list2)))): 

            while True: 
                try: 
                    tup = (list1[i], list2[i]) 
                except IndexError: 
                    if len(list1) > len(list2): 
                        list2.append('') 
                        tup = (list1[i], list2[i]) 
                    elif len(list1) < len(list2): 
                        list1.append('') 
                        tup = (list1[i], list2[i]) 
                    continue

                merged_list.append(tup) 
                break
        return merged_list 

In [115]:
authordict={}

authortuples=df[["authoridlist", "authorlist"]].apply(lambda x: mergelists(*x), axis=1)

for i in range(len(authortuples)): 
    try: 
        authordict.update(authortuples[i])
    except: 
        continue 

{'16438564300': 'Bonomi L.',
 '7005720033': ' Marchini G.',
 '6701396773': ' Marraffa M.',
 '57192481974': ' Bernardi P.',
 '6603499111': ' Morbio R.',
 '6603568658': ' Varotto A.',
 '6506659416': 'Tanure M.A.G.',
 '7403588976': ' Cohen E.J.',
 '6603349480': ' Sudesh S.',
 '7005117622': ' Rapuano C.J.',
 '24373593200': ' Laibson P.R.',
 '35398389300': 'Levy R.A.',
 '6506151613': ' Vilela V.S.',
 '57196699440': ' Cataldo M.J.',
 '55437940200': ' Ramos R.C.',
 '57206156107': ' Duarte J.L.M.B.',
 '57188640164': ' Tura B.R.',
 '7102992844': ' Albuquerque E.M.N.',
 '9840699100': ' Jesús N.R.',
 '26326720800': ' Bursell S.-E.',
 '6701580100': ' Cavallerano J.D.',
 '8906005300': ' Cavallerano A.',
 '7004750042': ' Clermont A.C.',
 '6602585121': ' Birkmire-Peters D.',
 '57208786099': ' Aiello L.P.',
 '7006805543': ' Aiello L.P.',
 '56704320200': ' Nichols J.J.',
 '6506548363': ' Marsich M.M.',
 '57212857874': ' Nguyen M.T.',
 '7202043696': ' Barr J.T.',
 '7006517768': ' Bullimore M.A.',
 '2443

In [116]:
df.columns
#so I haven't done anything with the author-id dictionary, haven't decided yet to use more granular author features 

Index(['index', 'authors', 'author(s) id', 'title', 'year', 'source title',
       'page start', 'page end', 'cited by', 'affiliations', 'abstract',
       'author keywords', 'index keywords', 'funding details',
       'correspondence address', 'access type', 'pagecount', 'fundingB',
       'open access', 'authorlist', 'authoridlist', 'authortotal'],
      dtype='object')

# Process Keywords (do this last due to memory constraints) 

In [121]:
indexkeywordlist=getcomponentwords(12)

In [147]:
len(indexkeywordlist)
indexkeywordlist

29305

['body postures',
 'los angeles',
 'isolation and purification',
 'viperinae',
 'straylight meter',
 'molecular model',
 'atlas-based methods',
 'stresses',
 'zimbabwe',
 'minimal additional efforts',
 'therapy',
 'patient rights',
 'gene order',
 'diabetes, gestational',
 'inhalation anesthesia',
 'grafts',
 'end-to-end solutions',
 'character recognition',
 'eyelid inflammation',
 'viruses',
 'graft detachment',
 'discrete wavelet transforms',
 'rhabdomyoma',
 'hydroxymethylglutaryl coenzyme a reductase inhibitor',
 'epithelial proliferation rate',
 'infrared illumination',
 'migraine like headaches',
 'tear film break up time',
 'nissl staining',
 'burn scar',
 'multiplex assay',
 'ross syndrome',
 'cerebral visual impairment',
 'cross-linking reagents',
 'network development',
 'squamous epithelium',
 'chromogenic in situ hybridization',
 'repeat donor keratoplasty',
 'nasal polyps',
 'dilution',
 'psychomotor performance',
 'serpentirhabdias specie',
 'a2-e (n-retinylidene-n-retin

In [123]:
authorkeywordlist=getcomponentwords(11)

In [124]:
len(authorkeywordlist)

13127

In [165]:
#takes a long time, as it's creating almost 30k new columns 
#only save columns with a decent amt of variance (3% threshold)
for word in indexkeywordlist: 
    newcolumn=df['index keywords'].str.count(word)
    if 0.03<=newcolumn.mean()<=0.97: 
        df["ik_"+word]=newcolumn 

In [166]:
df.columns
newcolumn.sum()

Index(['index', 'authors', 'author(s) id', 'title', 'year', 'source title',
       'page start', 'page end', 'cited by', 'affiliations',
       ...
       'ik_hemorrhage', 'ik_refractive error', 'ik_edema',
       'ik_cross-sectional study', 'ik_retinography', 'ik_sensitivity',
       'ik_ocular hypertension', 'ik_visual disorder',
       'ik_nuclear magnetic resonance', 'ik_segmentation'],
      dtype='object', length=456)

2.0

In [189]:
#should takea bout half as long 
for word in authorkeywordlist: 
    #deal with parentheses 
    word=word.replace('(', '\(')
    word=word.replace(')', '\)')
    newcolumn=df['author keywords'].str.count(word)
    if 0.03<=newcolumn.mean()<=0.97: 
        df["ak_"+word]=newcolumn

In [199]:
df.columns

Index(['index', 'authors', 'author(s) id', 'title', 'year', 'source title',
       'page start', 'page end', 'cited by', 'affiliations',
       ...
       'journal of aapos', 'journal of cataract and refractive surgery',
       'journal of glaucoma', 'journal of neuro-ophthalmology',
       'journal of pediatric ophthalmology and strabismus',
       'ophthalmic epidemiology', 'ophthalmology',
       'optometry and vision science', 'retina', 'survey of ophthalmology'],
      dtype='object', length=559)

# cleanup, train/test/split and save new datafile 

In [200]:
columnlist=["authors", "author(s) id", "title", "abstract", "source title", "page start", "page end", "affiliations", 
            "funding details", "author keywords", "index keywords", "correspondence address"]
for column in columnlist: 
    del df[column]
    

In [202]:
df.columns
df.dtypes

Index(['index', 'year', 'cited by', 'access type', 'pagecount', 'fundingB',
       'open access', 'authorlist', 'authoridlist', 'authortotal',
       ...
       'journal of aapos', 'journal of cataract and refractive surgery',
       'journal of glaucoma', 'journal of neuro-ophthalmology',
       'journal of pediatric ophthalmology and strabismus',
       'ophthalmic epidemiology', 'ophthalmology',
       'optometry and vision science', 'retina', 'survey of ophthalmology'],
      dtype='object', length=547)

index                             int64
year                              int64
cited by                        float64
access type                      object
pagecount                       float64
                                 ...   
ophthalmic epidemiology           uint8
ophthalmology                     uint8
optometry and vision science      uint8
retina                            uint8
survey of ophthalmology           uint8
Length: 547, dtype: object

In [203]:
df = df.rename(columns = {'cited by':'tc'}) 

In [205]:
#train/val/test split 
import random 
random.seed(1)
index=list(range(len(df))) #make this list as long as your dataset 
random.shuffle(index)
testindex=index[0:1000] #choose size of test and dev sets 
devindex=index[1000:2000]
trainindex=index[2000:]


In [221]:
df["split"]=0
df.loc[devindex,'split']=1
df.loc[testindex,'split']=2
df["split"].value_counts()

0    7392
2    1000
1    1000
Name: split, dtype: int64

In [220]:
df.loc[devindex,'split']=1

In [214]:
df["split"].value_counts()

0    9392
Name: split, dtype: int64

In [204]:
df.to_csv("dfscopus-2.csv", index=None)